In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/shim/dev/log-analytics


In [2]:
import torch
import numpy as np
import pandas as np
import pickle
from tqdm import tqdm
import sys

In [3]:
deck1 = torch.load("results/distilbert-base-uncased-ver6/distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_3-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver6/distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_3-deck2.pth")

In [4]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([421079, 768]), torch.Size([1095951, 768]))

In [5]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [6]:
deck1.keys(), deck2.keys()

(dict_keys(['feat', 'otext', 'tlevel', 'fclevel', 'feat_']),
 dict_keys(['feat', 'otext', 'fclevel', 'ids', 'feat_']))

In [7]:
N = deck2["feat"].size(0)
distdeck = {"dist": [], "level": [], "ids": []}
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][i, None], p=None, dim=1)
        dist_, indices_ = dist_.topk(4, largest=False)
        tlevels = deck1["tlevel"][indices_]

        distdeck["dist"].append(dist_.cpu())
        distdeck["level"].append(tlevels)
        distdeck["ids"].append(deck2["ids"][i])

        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [1:56:51<00:00, 156.30it/s]


In [8]:
distdeck["dist"] = torch.stack(distdeck["dist"])
distdeck["level"] = torch.stack(distdeck["level"])

In [10]:
torch.save(
    distdeck, "results/distilbert-base-uncased-ver6/distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_3-distdeck.pth"
)